In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import geopandas as gpd
from shapely.geometry import MultiPoint, Polygon # Importing MultiPoint and Polygon

In [ ]:
#loading the dataset
df=pd.read_csv('/content/drive/MyDrive/data-visualization/Police_Department_Incidents_-_Previous_Year__2016_.csv')
#loading the dataset
df

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150495,161061000,ASSAULT,BATTERY,Friday,12/30/2016 12:00:00 AM,21:01,PARK,NONE,OAK ST / STANYAN ST,-122.453982,37.771428,"(37.7714278595913, -122.453981622365)",16106100004134
150496,176000742,NON-CRIMINAL,LOST PROPERTY,Friday,12/30/2016 12:00:00 AM,08:00,CENTRAL,NONE,JACKSON ST / SANSOME ST,-122.401857,37.796626,"(37.7966261239618, -122.401857374739)",17600074271000
150497,176000758,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Thursday,12/29/2016 12:00:00 AM,20:00,CENTRAL,NONE,PINE ST / TAYLOR ST,-122.412269,37.790673,"(37.7906727649886, -122.41226909106)",17600075806372
150498,176000764,LARCENY/THEFT,GRAND THEFT OF PROPERTY,Friday,12/30/2016 12:00:00 AM,10:00,CENTRAL,NONE,200 Block of STOCKTON ST,-122.406659,37.788275,"(37.7882745285785, -122.406658711008)",17600076406374


In [ ]:
import random

# Chọn các loại tội phạm nghiêm trọng nhất (ví dụ: Bạo lực, Vũ khí, Ma túy)
important_categories = ["ASSAULT", "ROBBERY", "BURGLARY", "WEAPON LAWS", "KIDNAPPING", "SEX OFFENSES, FORCIBLE"]

# Assuming 'df' is your original DataFrame
df_valid = df.dropna(subset=['PdDistrict', 'X', 'Y']) # Define df_valid here

# Lọc dữ liệu theo các loại tội phạm nghiêm trọng
df_important = df_valid[df_valid["Category"].isin(important_categories)]

# Chọn ngẫu nhiên 20 vụ việc quan trọng
df_sample = df_important.sample(n=20, random_state=42)

# Tạo bản đồ
# Assuming 'sf_center' is defined and contains the center coordinates for your map
m = folium.Map(location=[37.7749, -122.4194], zoom_start=12) # Replace with your actual sf_center

# Thêm các điểm quan trọng lên bản đồ
for _, row in df_sample.iterrows():
    lat, lon = row["Y"], row["X"]
    category = row["Category"]
    description = row["Descript"]
    date, time = row["Date"], row["Time"]
    address = row["Address"]

    popup_text = f"<b>{category}</b><br>{description}<br><b>Date:</b> {date} {time}<br><b>Address:</b> {address}"

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(m)

# Xuất bản đồ
m

In [ ]:
import pandas as pd
import folium
from scipy.spatial import ConvexHull
import numpy as np


# Lọc dữ liệu tọa độ hợp lệ
coords = df[['Y', 'X']].dropna().values  # Đổi thứ tự để phù hợp với bản đồ (lat, lon)

# Tính toán Convex Hull
hull = ConvexHull(coords)

# Tạo bản đồ tập trung vào trung tâm dữ liệu
center = np.mean(coords, axis=0)
map_obj = folium.Map(location=center, zoom_start=12)

# Thêm đường biên Convex Hull
hull_points = [coords[i] for i in hull.vertices]
hull_points.append(hull_points[0])  # Đóng đường biên
folium.PolyLine(hull_points, color='red', weight=2.5, opacity=0.8).add_to(map_obj)

# Hiển thị bản đồ
map_obj


In [ ]:


# Lọc dữ liệu tọa độ hợp lệ
coords = df[['Y', 'X']].dropna().values  # Đổi thứ tự để phù hợp với bản đồ (lat, lon)

# Tính toán Convex Hull
hull = ConvexHull(coords)

# Tạo bản đồ tập trung vào trung tâm dữ liệu
center = np.mean(coords, axis=0)
map_obj = folium.Map(location=center, zoom_start=12)

# Thêm đường biên Convex Hull
hull_points = [coords[i] for i in hull.vertices]
hull_points.append(hull_points[0])  # Đóng đường biên
folium.PolyLine(hull_points, color='red', weight=2.5, opacity=0.8).add_to(map_obj)

# Chọn 20 vụ án lớn nhất theo mức độ nghiêm trọng (giả định dựa trên Category và Resolution)
important_cases = df.sort_values(by=['Category', 'Resolution'], ascending=False).head(20)

# Thêm các vụ án lên bản đồ
for _, row in important_cases.iterrows():
    folium.Marker(
        location=[row['Y'], row['X']],
        popup=f"{row['Category']}: {row['Descript']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_obj)

# Hiển thị bản đồ
map_obj


In [ ]:

# Lọc dữ liệu tọa độ hợp lệ
coords = df[['Y', 'X']].dropna().values  # Đổi thứ tự để phù hợp với bản đồ (lat, lon)

# Tính toán Convex Hull
hull = ConvexHull(coords)

# Tạo bản đồ tập trung vào trung tâm dữ liệu
center = np.mean(coords, axis=0)
map_obj = folium.Map(location=center, zoom_start=12)

# Thêm đường biên Convex Hull
hull_points = [coords[i] for i in hull.vertices]
hull_points.append(hull_points[0])  # Đóng đường biên
folium.PolyLine(hull_points, color='red', weight=2.5, opacity=0.8).add_to(map_obj)

# Chọn 20 vụ án lớn nhất theo mức độ nghiêm trọng (giả định dựa trên Category và Resolution)
important_cases = df.sort_values(by=['Category', 'Resolution'], ascending=False).head(20)

# Thêm các vụ án lên bản đồ
for _, row in important_cases.iterrows():
    folium.Marker(
        location=[row['Y'], row['X']],
        popup=f"ID: {row['IncidntNum']}<br>Category: {row['Category']}<br>Description: {row['Descript']}<br>Date: {row['Date']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_obj)

# Hiển thị bản đồ
map_obj


In [ ]:

# Lọc dữ liệu tọa độ hợp lệ
coords = df[['Y', 'X']].dropna().values  # Đổi thứ tự để phù hợp với bản đồ (lat, lon)

# Tính toán Convex Hull
hull = ConvexHull(coords)

# Tạo bản đồ tập trung vào trung tâm dữ liệu
center = np.mean(coords, axis=0)
map_obj = folium.Map(location=center, zoom_start=12)

# Thêm đường biên Convex Hull
hull_points = [coords[i] for i in hull.vertices]
hull_points.append(hull_points[0])  # Đóng đường biên
folium.PolyLine(hull_points, color='red', weight=2.5, opacity=0.8).add_to(map_obj)

# Lấy ngẫu nhiên 20 vụ án
random_cases = df.sample(n=20, random_state=42)

# Thêm các vụ án lên bản đồ
for _, row in random_cases.iterrows():
    folium.Marker(
        location=[row['Y'], row['X']],
        popup=f"ID: {row['IncidntNum']}<br>Category: {row['Category']}<br>Description: {row['Descript']}<br>Date: {row['Date']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_obj)

# Hiển thị bản đồ
map_obj


In [ ]:


# Xác định quận có nhiều vụ án nhất
most_common_district = df['PdDistrict'].value_counts().idxmax()
district_df = df[df['PdDistrict'] == most_common_district]
num_cases = len(district_df)

# Lọc dữ liệu tọa độ hợp lệ trong quận đó
coords = district_df[['Y', 'X']].dropna().values  # Đổi thứ tự để phù hợp với bản đồ (lat, lon)

# Tính toán Convex Hull cho quận có nhiều vụ án nhất
hull = ConvexHull(coords)

# Tạo bản đồ tập trung vào quận đó
center = np.mean(coords, axis=0)
map_obj = folium.Map(location=center, zoom_start=12)

# Thêm đường biên Convex Hull cho quận đó
hull_points = [coords[i] for i in hull.vertices]
hull_points.append(hull_points[0])  # Đóng đường biên
folium.PolyLine(hull_points, color='red', weight=2.5, opacity=0.8).add_to(map_obj)

# Hiển thị thông tin của quận
info_popup = folium.Marker(
    location=center,
    popup=f"District: {most_common_district}<br>Total Cases: {num_cases}",
    icon=folium.Icon(color='green', icon='info-sign')
)
info_popup.add_to(map_obj)

# Lấy ngẫu nhiên 20 vụ án trong quận đó
random_cases = district_df.sample(n=20, random_state=42)

# Thêm các vụ án lên bản đồ
for _, row in random_cases.iterrows():
    folium.Marker(
        location=[row['Y'], row['X']],
        popup=f"ID: {row['IncidntNum']}<br>Category: {row['Category']}<br>Description: {row['Descript']}<br>Date: {row['Date']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_obj)

# Hiển thị bản đồ
map_obj
